In [ ]:
import os
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset

class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths, self.labels = self.process_images(root_dir)
        self.classes = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.classes.index(self.labels[idx])
        
        if self.transform:
            image = self.transform(image)
        return image, label

    def process_images(self, path):
        images = []
        labels = []
        for category in os.listdir(path):
            category_path = os.path.join(path, category)
            if os.path.isdir(category_path):
                for img_name in os.listdir(category_path):
                    img_path = os.path.join(category_path, img_name)
                    if self.is_image_file(img_path):
                        images.append(img_path)
                        labels.append(category)
        return images, labels

    def is_image_file(self, filename):
        valid_image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]
        return any(filename.lower().endswith(ext) for ext in valid_image_extensions)

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Less aggressive cropping
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),  # Reduce rotation angle
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  # Less aggressive color jitter
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

In [ ]:
class BYOL(nn.Module):
    def __init__(self, base_model, projection_size=256, projection_hidden_size=4096):
        super(BYOL, self).__init__()
        self.online_encoder = base_model
        self.target_encoder = base_model

        self.projector = nn.Sequential(
            nn.Linear(1000, projection_hidden_size),
            nn.BatchNorm1d(projection_hidden_size),
            nn.ReLU(),
            nn.Linear(projection_hidden_size, projection_size)
        )

    def forward(self, x):
        online_proj = self.projector(self.online_encoder(x))
        with torch.no_grad():
            target_proj = self.projector(self.target_encoder(x))
        return online_proj, target_proj

class Classifier(nn.Module):
    def __init__(self, base_model, num_classes=4):
        super(Classifier, self).__init__()
        self.backbone = base_model
        self.fc = nn.Linear(1000, num_classes)
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x)

In [ ]:
# Function to fine-tune the model
def fine_tune_model(classifier_model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    # Unfreeze all layers in the backbone model
    for param in classifier_model.backbone.parameters():
        param.requires_grad = True

    # Re-initialize optimizer to update all parameters
    optimizer = optim.Adam(classifier_model.parameters(), lr=1e-4)

    # Fine-tune the model
    train_model(classifier_model, train_loader, val_loader, criterion, optimizer, num_epochs, device)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

byol_model = BYOL(base_model).to(device)
byol_optimizer = optim.Adam(byol_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


classifier_model = Classifier(base_model).to(device)

for param in classifier_model.backbone.parameters():
    param.requires_grad = False

classifier_optimizer = optim.Adam(classifier_model.parameters(), lr=1e-3)
fine_tune_model(classifier_model, train_loader, val_loader, criterion, classifier_optimizer, num_epochs=30, device=device)